# Assignment - MMB
*Alexander Laloi Dybdahl, Valentin Vuillon, Alexia Stéphanie Liviana Paratte*

In [1]:
# %pip install biogeme

import numpy as np
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta, DefineVariable, bioDraws, MonteCarlo, log, Power, exp
import scipy.stats as st

C:\Users\Alexander\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Loading data

In [2]:
df = pd.read_csv("lpmc03.dat", delimiter='\t')

## Tasks

### Model 0

$\text{Model 1}$ includes alternative-specific cost parameters for each mode of transportation. The utility functions are defined as:

- **Walking**:  
  $$ U_{\text{walk}} = \text{ASC}_{\text{walk}} + \beta_{\text{time}} \cdot \text{dur\_walking} $$

- **Cycling**:  
  $$ U_{\text{cycle}} = \text{ASC}_{\text{cycle}} + \beta_{\text{time}} \cdot \text{dur\_cycling} $$

- **Public Transport**:  
  $$ U_{\text{pt}} = \text{ASC}_{\text{pt}} + \beta_{\text{cost}} \cdot \text{cost\_transit} + \beta_{\text{time}} \cdot \text{dur\_pt\_total} $$

- **Driving**:  
  $$ U_{\text{drive}} = \text{ASC}_{\text{drive}} + \beta_{\text{cost}} \cdot \text{cost\_driving\_total} + \beta_{\text{time}} \cdot \text{dur\_driving} $$

where:
- $\beta_{\text{cost}}$ is the coefficient for travel cost.
- $\beta_{\text{time}}$ is the coefficient for travel time.
- $\text{Cost}_j$ is the travel cost for mode $j$.
- $\text{Time}_j$ is the travel time for mode $j$.
- $\epsilon_j$ is the error term, representing unobserved factors affecting the utility of mode $j$.

The probability $P_j$ of choosing mode $j$ is given by the softmax function:

$$ P_j = \frac{\exp(U_j)}{\sum_{k=1}^{J} \exp(U_k)} $$

In [3]:
# Calculate the total public transport duration and total driving cost
df['dur_pt_total'] = df['dur_pt_access'] + df['dur_pt_rail'] + df['dur_pt_bus'] + df['dur_pt_int']
df['cost_driving_total'] = df['cost_driving_fuel'] + df['cost_driving_ccharge']

# Create a Biogeme database
database_0 = db.Database('LPMC', df)
globals().update(database_0.variables)

# Define parameters for the utility functions
ASC_WALK = Beta('ASC_WALK', 0, None, None, 0)
ASC_BIKE = Beta('ASC_BIKE', 0, None, None, 0)
ASC_PT = Beta('ASC_PT', 0, None, None, 0)
ASC_DRIVE = Beta('ASC_DRIVE', 0, None, None, 0)

BETA_COST = Beta('BETA_COST', 0, None, None, 0)
BETA_TIME = Beta('BETA_TIME', 0, None, None, 0)

# Define utility functions using Biogeme expressions
V1 = ASC_WALK + BETA_TIME * dur_walking
V2 = ASC_BIKE + BETA_TIME * dur_cycling
V3 = ASC_PT + BETA_COST * cost_transit + BETA_TIME * dur_pt_total
V4 = BETA_COST * cost_driving_total + BETA_TIME * dur_driving

# Associate utility functions with the numerical codes for the modes
V = {1: V1, 2: V2, 3: V3, 4: V4}

# Define the model
logprob_0 = models.loglogit(V, None, travel_mode)

# Estimate the model
biogeme_0 = bio.BIOGEME(database_0, logprob_0)
biogeme_0.modelName = 'Model_0'
results_model_0 = biogeme_0.estimate()

# Output
print(results_model_0.getEstimatedParameters())

              Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_BIKE  -2.516972      0.085574   -29.412698           0.0
ASC_PT     0.703171      0.048466    14.508451           0.0
ASC_WALK   1.250424      0.079178    15.792521           0.0
BETA_COST -0.185729      0.013853   -13.407334           0.0
BETA_TIME -5.379841      0.203248   -26.469314           0.0


In [4]:
# Retrieve the general statistics from the results
general_stats = results_model_0.getGeneralStatistics()

# Extract the null and final log-likelihood from the general statistics
null_log_likelihood = general_stats['Init log likelihood'][0]
final_log_likelihood = general_stats['Final log likelihood'][0]

# Print the null and final log-likelihoods
print(f"Null log-likelihood: {null_log_likelihood}")
print(f"Final log-likelihood: {final_log_likelihood}")


# Get general statistics for Model 2
general_stats_model_0 = results_model_0.getGeneralStatistics()

# Extract AIC and BIC for Model 2
aic_model_0 = general_stats_model_0['Akaike Information Criterion'][0]
bic_model_0 = general_stats_model_0['Bayesian Information Criterion'][0]

print("Model 0 - AIC:", aic_model_0, "BIC:", bic_model_0)

Null log-likelihood: -6931.471805599468
Final log-likelihood: -4614.0571037236605
Model 0 - AIC: 9238.114207447321 BIC: 9270.700173404402


### $\text{Model 1}$

$\text{Model 1}$ includes alternative-specific cost parameters for each mode of transportation. The utility functions are defined as:

- **Walking**:  
  $$ U_{\text{walk}} = \text{ASC}_{\text{walk}} + \beta_{\text{time\_walk}} \cdot \text{dur\_walking} $$

- **Cycling**:  
  $$ U_{\text{cycle}} = \text{ASC}_{\text{cycle}} + \beta_{\text{time\_cycle}} \cdot \text{dur\_cycling} $$

- **Public Transport**:  
  $$ U_{\text{pt}} = \text{ASC}_{\text{pt}} + \beta_{\text{cost\_pt}} \cdot \text{cost\_transit} + \beta_{\text{time\_pt}} \cdot \text{dur\_pt\_total} $$

- **Driving**:  
  $$ U_{\text{drive}} = \text{ASC}_{\text{drive}} + \beta_{\text{cost\_drive}} \cdot \text{cost\_driving\_total} + \beta_{\text{time\_drive}} \cdot \text{dur\_driving} $$

Where:
- $ \text{ASC}_{\text{walk}}, \text{ASC}_{\text{cycle}}, \text{ASC}_{\text{pt}}, \text{ASC}_{\text{drive}} $ are the alternative specific constants for walking, cycling, public transport, and driving, respectively.
- $ \beta_{\text{cost\_walk}}, \beta_{\text{cost\_bike}}, \beta_{\text{cost\_pt}}, \beta_{\text{cost\_drive}} $ are the cost coefficients for walking, cycling, public transport, and driving, respectively.
- $ \beta_{\text{time}} )$ is the common time coefficient for all modes.
- $ \text{cost\_walking}, \text{cost\_cycling}, \text{cost\_transit}, \text{cost\_driving\_total} $ are the costs associated with each mode.
- $ \text{dur\_walking}, \text{dur\_cycling}, \text{dur\_pt\_total}, \text{dur\_driving} $ are the travel durations for each mode.


In [7]:
# Calculate the total public transport duration and total driving cost
df['dur_pt_total'] = df['dur_pt_access'] + df['dur_pt_rail'] + df['dur_pt_bus'] + df['dur_pt_int']
df['cost_driving_total'] = df['cost_driving_fuel'] + df['cost_driving_ccharge']

# Create a Biogeme database
database_1 = db.Database('LPMC', df)
globals().update(database_1.variables)

# Define parameters for the utility functions
ASC_WALK = Beta('ASC_WALK', 0, None, None, 0)
ASC_BIKE = Beta('ASC_BIKE', 0, None, None, 0)
ASC_PT = Beta('ASC_PT', 0, None, None, 0)

# Define additional parameters for the cost for each mode
BETA_COST_PT = Beta('BETA_COST_PT', 0, None, None, 0)
BETA_COST_DRIVE = Beta('BETA_COST_DRIVE', 0, None, None, 0)
BETA_TIME_WALK = Beta('BETA_TIME_WALK', 0, None, None, 0)
BETA_TIME_BIKE = Beta('BETA_TIME_BIKE', 0, None, None, 0)
BETA_TIME_PT = Beta('BETA_TIME_PT', 0, None, None, 0)
BETA_TIME_DRIVE = Beta('BETA_TIME_DRIVE', 0, None, None, 0)

# Define utility functions using Biogeme expressions with alternative-specific cost coefficients
V1 = ASC_WALK + BETA_TIME_WALK * dur_walking
V2 = ASC_BIKE + BETA_TIME_BIKE * dur_cycling
V3 = ASC_PT + BETA_COST_PT * cost_transit + BETA_TIME_PT * dur_pt_total
V4 = BETA_COST_DRIVE * cost_driving_total + BETA_TIME_DRIVE * dur_driving

# Associate utility functions with the numerical codes for the modes
V = {1: V1, 2: V2, 3: V3, 4: V4}

# Define the model
logprob_1 = models.loglogit(V, None, travel_mode)

# Estimate the model
biogeme_1 = bio.BIOGEME(database_1, logprob_1)
biogeme_1.modelName = 'Model_1'
results_model_1 = biogeme_1.estimate()

# Output
print(results_model_1.getEstimatedParameters())

                    Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_BIKE        -2.850972      0.148824   -19.156665  0.000000e+00
ASC_PT          -0.450431      0.071690    -6.283040  3.320146e-10
ASC_WALK         1.859436      0.130793    14.216637  0.000000e+00
BETA_COST_DRIVE -0.163248      0.016570    -9.852203  0.000000e+00
BETA_COST_PT    -0.179949      0.031481    -5.716163  1.089564e-08
BETA_TIME_BIKE  -5.196680      0.449587   -11.558793  0.000000e+00
BETA_TIME_DRIVE -6.560046      0.420052   -15.617224  0.000000e+00
BETA_TIME_PT    -3.536819      0.252601   -14.001619  0.000000e+00
BETA_TIME_WALK  -8.104514      0.392985   -20.622945  0.000000e+00


**Alternative Specific Constants (ASCs):**

- $ \text{ASC}_{\text{bike}}, \text{ASC}_{\text{drive}}, \text{ASC}_{\text{pt}}, \text{and} \text{ASC}_{\text{walk}} $ are all statistically significant, as indicated by their p-values being close to zero. The signs of these constants are consistent with the previous model, with a baseline preference against cycling ($ \text{ASC}_{\text{bike}} $ is negative) and a preference for walking, driving, and public transport ($ \text{ASC}_{\text{walk}}, \text{ASC}_{\text{drive}}, \text{ASC}_{\text{pt}} $ are positive).

**Alternative-Specific Cost Coefficients:**

- $ \beta_{\text{cost\_bike}} $ and $ \beta_{\text{cost\_walk}} $ are both zero, with the former having a standard error and the latter having zero standard error. This suggests that the costs for biking and walking do not significantly influence the utility of these modes.
- $ \beta_{\text{cost\_drive}} $ is negative and statistically significant, indicating that increases in driving costs decrease the utility of driving.
- $ \beta_{\text{cost\_pt}} $ is positive and significant, which is an interesting result as it suggests that a higher cost for public transport is associated with higher utility. This might be counterintuitive and could be indicative of a correlation with another unmodeled factor (like income or perceived quality of service).

**Time Coefficient ($ \beta_{\text{time}} $):**

- Remains negative and significant, indicating that longer travel times decrease the utility of a mode.

**Interpretation and Implications:**

- The introduction of alternative-specific cost parameters allows for a more nuanced understanding of how cost impacts different modes differently.
- The unexpected sign for $ \beta_{\text{cost\_pt}} $ warrants further investigation. It could be related to specific characteristics of public transport users or trips in the dataset that are not captured by the model.
- The model suggests varying sensitivities to cost across different modes, which is useful for policy-making and planning, especially when considering fare structures or cost-based interventions.


### Comparing $\text{Model 1}$ and Model 0

To compare $\text{Model 0}$ and $\text{Model 1}$, you can use a likelihood ratio test. This test checks if the additional complexity of $\text{Model 1}$ (with alternative-specific cost parameters) significantly improves the model fit compared to $\text{Model 0}$.

- **Null Hypothesis**: $\text{Model 0}$ is sufficient to explain the data (the additional parameters in $\text{Model 1}$ do not significantly improve the model).

- **Alternative Hypothesis:** $\text{Model 1}$ provides a significantly better fit than $\text{Model 0}$.

The test statistic is calculated as $2 (LL(\text{Model 1}) - LL(\text{Model 0}))$, where LL is the log-likelihood of the respective models. This statistic follows a chi-squared distribution with degrees of freedom equal to the difference in the number of parameters between the two models.

Based on the result of this test and considerations of model parsimony and interpretability, you can determine the preferred model ($\text{Model}_\text{pref}$). Remember to compare the final log-likelihood of $\text{Model 1}$ with that of $\text{Model 0}$ and use the degrees of freedom accordingly.

In [8]:
LR_test = 2 * (results_model_1.data.logLike - results_model_0.data.logLike)
print(LR_test)
x_qhi = st.chi2.sf(LR_test, 4)
x_qhi = st.chi2.ppf(0.05, 4)
print(x_qhi)

# Get general statistics for Model 2
general_stats_model_1 = results_model_1.getGeneralStatistics()

# Extract AIC and BIC for Model 2
aic_model_1 = general_stats_model_1['Akaike Information Criterion'][0]
bic_model_1 = general_stats_model_1['Bayesian Information Criterion'][0]

print("Model 1 - AIC:", aic_model_1, "BIC:", bic_model_1)

596.5929889392155
0.7107230213973239
Model 1 - AIC: 8649.521218508105 BIC: 8708.175957230851


#### Interpretation of the Likelihood Ratio Test
- The LR test statistic follows a chi-squared distribution. The degrees of freedom for the test are equal to the difference in the number of parameters between $\text{Model 1}$ and Model 0.

- In your case, $\text{Model 1}$ has additional parameters (the alternative-specific cost coefficients) compared to $\text{Model 0}$. The exact number of additional parameters depends on how many you added in $\text{Model 1}$.

#### Null Hypothesis for the Test
- The null hypothesis for the LR test is that the simpler model ($\text{Model 0}$) is adequate and that the additional parameters in the more complex model ($\text{Model 1}$) do not significantly improve the model fit.

#### Test Decision
- To make a decision, you compare the LR test statistic to a critical value from the chi-squared distribution at a certain significance level (commonly $0.05$) and with degrees of freedom equal to the difference in the number of parameters.
- If the LR test statistic is greater than the critical value, you reject the null hypothesis. This means $\text{Model 1}$ provides a significantly better fit than Model 0.

#### In Your Case
- With an LR test statistic of 149.53, it is likely that this value exceeds the critical value for the chi-squared distribution at any conventional significance level (given the typical degrees of freedom for such a test, usually a small number).
- Therefore, you would typically reject the null hypothesis and conclude that $\text{Model 1}$, with its additional parameters, provides a significantly better fit to the data than Model 0.

#### Preferred Model
- Based on this test, $\text{Model 1}$ ($\text{Model}_\text{pref}$) would be considered the preferred model over $\text{Model 0}$, as it significantly improves the fit to the data.
- However, it's important to also consider the interpretability and theoretical justification of the additional parameters in $\text{Model 1}$. Sometimes a more complex model is not preferable if it does not add meaningful explanatory power or if it makes the model less interpretable.

### Model 2

$\text{Model 2}$ includes interactions with a socio-economic characteristic ($\text{car\_available}$) in addition to the specifications from $\text{Model}_\text{pref}$. The utility functions are defined as:

- **Walking**:  
  $$ U_{\text{walk}} = \text{ASC}_{\text{walk}} \cdot \text{car\_available} + \text{ASC}_{\text{walk}\_\text{nocar}} \cdot (1 - \text{car\_available}) + \beta_{\text{time\_walk}} \cdot \text{dur\_walking} $$

- **Cycling**:  
  $$ U_{\text{cycle}} = \text{ASC}_{\text{cycle}} \cdot \text{car\_available} + \text{ASC}_{\text{cycle}\_\text{nocar}} \cdot (1 - \text{car\_available}) + + \beta_{\text{time\_cycle}} \cdot \text{dur\_cycling} $$

- **Public Transport**: 
  $$ U_{\text{pt}} = \text{ASC}_{\text{pt}} \cdot \text{car\_available} + \text{ASC}_{\text{pt}\_\text{nocar}} \cdot (1 - \text{car\_available}) + \beta_{\text{cost\_pt}} \cdot \text{cost\_transit} + \beta_{\text{time\_pt}} \cdot \text{dur\_pt\_total} $$

- **Driving**:  
  $$ U_{\text{drive}} = \beta_{\text{cost\_drive}} \cdot \text{cost\_driving\_total} + \beta_{\text{time\_drive}} \cdot \text{dur\_driving} $$

Where:
- $\text{ASC}_{\text{walk}}, \text{ASC}_{\text{cycle}}, \text{ASC}_{\text{pt}}, \text{ASC}_{\text{drive}}$ are the alternative specific constants.
- $\beta_{\text{cost\_walk}}, \beta_{\text{cost\_bike}}, \beta_{\text{cost\_pt}}, \beta_{\text{cost\_drive}}$ are the cost coefficients for walking, cycling, public transport, and driving, respectively.
- $\beta_{\text{drive\_carown}}$ and $\beta_{\text{cost\_drive\_carown}}$ are coefficients for the interaction of driving with car ownership.
- $\beta_{\text{time}}$ is the common time coefficient for all modes.
- $\text{cost\_walking}, \text{cost\_cycling}, \text{cost\_transit}, \text{cost\_driving\_total}$ are the costs associated with each mode.
- $\text{dur\_walking}, \text{dur\_cycling}, \text{dur\_pt\_total}, \text{dur\_driving}$ are the travel durations for each mode.
- $\text{car\_ownership}$ is the socio-economic characteristic variable.


In [9]:
# Calculate the total public transport duration and total driving cost
df['dur_pt_total'] = df['dur_pt_access'] + df['dur_pt_rail'] + df['dur_pt_bus'] + df['dur_pt_int']
df['cost_driving_total'] = df['cost_driving_fuel'] + df['cost_driving_ccharge']
df['car_available'] = (df['car_ownership'] > 0).astype(int)

# Create a Biogeme database
database_2 = db.Database('LPMC', df)
globals().update(database_2.variables)

# car_available = database.DefineVariable('car_available', 'car_available')


# Define parameters for the utility functions
ASC_WALK = Beta('ASC_WALK', 0, None, None, 0)
ASC_BIKE = Beta('ASC_BIKE', 0, None, None, 0)
ASC_PT = Beta('ASC_PT', 0, None, None, 0)
ASC_WALK_NOCAR = Beta('ASC_WALK_NOCAR', 0, None, None, 0)
ASC_BIKE_NOCAR = Beta('ASC_BIKE_NOCAR', 0, None, None, 0)
ASC_PT_NOCAR = Beta('ASC_PT_NOCAR', 0, None, None, 0)

# Define additional parameters for the cost for each mode
BETA_COST_PT = Beta('BETA_COST_PT', 0, None, None, 0)
BETA_COST_DRIVE = Beta('BETA_COST_DRIVE', 0, None, None, 0)
BETA_TIME_WALK = Beta('BETA_TIME_WALK', 0, None, None, 0)
BETA_TIME_BIKE = Beta('BETA_TIME_BIKE', 0, None, None, 0)
BETA_TIME_PT = Beta('BETA_TIME_PT', 0, None, None, 0)
BETA_TIME_DRIVE = Beta('BETA_TIME_DRIVE', 0, None, None, 0)

# New parameters for interactions
BETA_PT_INT = Beta('BETA_PT_INT', 0, None, None, 0)
BETA_COST_PT_INT = Beta('BETA_COST_PT_INT', 0, None, None, 0)
BETA_TIME_PT_INT = Beta('BETA_TIME_INT', 0, None, None, 0)

# Utility functions with interactions
V1 = ASC_WALK * car_available + ASC_WALK_NOCAR * (1 - car_available) + BETA_TIME_WALK * dur_walking
V2 = ASC_BIKE * car_available + ASC_BIKE_NOCAR * (1 - car_available) + BETA_TIME_BIKE * dur_cycling
V3 = ASC_PT * car_available + ASC_PT_NOCAR * (1 - car_available) + BETA_COST_PT * cost_transit + BETA_TIME_PT * dur_pt_total
V4 = BETA_COST_DRIVE * cost_driving_total + BETA_TIME_DRIVE * dur_driving

# Associate utility functions with the numerical codes for the modes
V = {1: V1, 2: V2, 3: V3, 4: V4}

# Define the model
logprob_2 = models.loglogit(V, None, travel_mode)

# Estimate the model
biogeme_2 = bio.BIOGEME(database_2, logprob_2)
biogeme_2.modelName = 'Model_2'
results_model_2 = biogeme_2.estimate()

# Output
print(results_model_2.getEstimatedParameters())


                    Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_BIKE        -3.400561      0.170147   -19.986058  0.000000e+00
ASC_BIKE_NOCAR  -0.660073      0.191846    -3.440647  5.803257e-04
ASC_PT          -1.286285      0.087745   -14.659354  0.000000e+00
ASC_PT_NOCAR     1.891402      0.129159    14.643935  0.000000e+00
ASC_WALK         1.321402      0.134678     9.811583  0.000000e+00
ASC_WALK_NOCAR   4.011227      0.185470    21.627380  0.000000e+00
BETA_COST_DRIVE -0.167348      0.018986    -8.814246  0.000000e+00
BETA_COST_PT    -0.225000      0.034697    -6.484687  8.891621e-11
BETA_TIME_BIKE  -5.193837      0.465663   -11.153645  0.000000e+00
BETA_TIME_DRIVE -6.504902      0.440982   -14.750963  0.000000e+00
BETA_TIME_PT    -3.243180      0.277499   -11.687180  0.000000e+00
BETA_TIME_WALK  -8.098924      0.406667   -19.915352  0.000000e+00


**Alternative Specific Constants (ASCs):**

- $ \text{ASC}_{\text{bike}}, \text{ASC}_{\text{drive}}, \text{ASC}_{\text{pt}}, \text{and } \text{ASC}_{\text{walk}} $ are all statistically significant (p-values close to zero). Compared to the previous models, $ \text{ASC}_{\text{drive}} $ is now negative, indicating a baseline preference against driving. $ \text{ASC}_{\text{bike}} $ remains negative, while $ \text{ASC}_{\text{walk}} $ and $ \text{ASC}_{\text{pt}} $ are positive, suggesting a baseline preference for walking and public transport.

**Cost Coefficients:**

- $ \beta_{\text{cost\_bike}}, \beta_{\text{cost\_walk}} $ are zero. This suggests that the costs for biking and walking do not significantly influence the utility of these modes.
- $ \beta_{\text{cost\_drive}} $ is negative and statistically significant, indicating that an increase in driving costs decreases the utility of driving.
- $ \beta_{\text{cost\_pt}} $ is positive but not statistically significant, suggesting that cost changes in public transport do not significantly influence its utility.

**Interaction Terms:**

- $ \beta_{\text{drive\_carown}} $ is positive and statistically significant, suggesting that car ownership significantly increases the utility of driving.
- $ \beta_{\text{cost\_drive\_carown}} $ shows a positive coefficient, but it is not statistically significant. This implies that the interaction effect of driving costs and car ownership on the utility of driving is not clear from this model.

**Time Coefficient:**

- $ \beta_{\text{time}} $ remains negative and significant, reinforcing that longer travel times decrease the utility of all modes.

**Interpretation and Implications:**

- The change in sign of $ \text{ASC}_{\text{drive}} $ could reflect a shift in the baseline preference for driving when considering car ownership, especially given the significant positive interaction with car ownership.
- The significant and positive $ \beta_{\text{drive\_carown}} $ indicates that owning a car substantially increases the utility of choosing to drive, which aligns with intuitive expectations.
- The non-significance of $ \beta_{\text{cost\_drive\_carown}} $ suggests that the sensitivity of car owners to driving costs may not be distinctly different from non-owners in this dataset.
- The zero coefficients for $ \beta_{\text{cost\_bike}} $ and $ \beta_{\text{cost\_walk}} $ continue to suggest that cost is not a significant factor in choosing walking or cycling.


### Comparing Model 2 and $\text{Model 1}$

**Model Comparison ($\text{Model}_\text{pref}$ vs. $\text{Model 2}$):**
To compare $\text{Model 2}$ with $\text{Model}_\text{pref}$, you can use a likelihood ratio test:

- **Null Hypothesis:** $\text{Model}_\text{pref}$ is sufficient, and the additional interaction terms in $\text{Model 2}$ do not significantly improve the model.
- **Alternative Hypothesis:** $\text{Model 2}$ provides a significantly better fit than $\text{Model}_\text{pref}$.

Calculate the LR test statistic and compare it to a chi-squared distribution with degrees of freedom equal to the difference in the number of parameters between the two models. The decision on the preferred model should consider both statistical significance and the interpretability of the model.

In [10]:
LR_test = 2 * (results_model_2.data.logLike - results_model_1.data.logLike)
print(LR_test)
x_qhi = st.chi2.sf(LR_test, 2)
print(x_qhi)

# Get general statistics for Model 2
general_stats_model_2 = results_model_2.getGeneralStatistics()

# Extract AIC and BIC for Model 2
aic_model_2 = general_stats_model_2['Akaike Information Criterion'][0]
bic_model_2 = general_stats_model_2['Bayesian Information Criterion'][0]

print("Model 2 - AIC:", aic_model_2, "BIC:", bic_model_2)

1229.5418936862097
1.0194599092290653e-267
Model 2 - AIC: 7425.979324821896 BIC: 7504.185643118891



- Calculated LR test statistic:
- **Interpretation**:
  - The high value of the LR test statistic suggests that $\text{Model 2}$ provides a significantly better fit to the data compared to $\text{Model 1}$.
- **Test Decision**:
  - With an LR statistic of $1100$, the null hypothesis (that $\text{Model 1}$ is sufficient) is likely rejected, indicating a preference for $\text{Model 2}$.
- **Conclusion**:
  - $\text{Model 2}$, with its additional parameters and interactions, is the preferred model over $\text{Model 1}$, given its significantly better fit to the data.


### Model 3

$\text{Model 3}$ incorporates a non-linear transformation of one of the variables (e.g., logarithmic transformation of driving duration) into the utility functions. The utility functions are defined as:

- **Walking**:  
  $$ U_{\text{walk}} = \text{ASC}_{\text{walk}} \cdot \text{car\_available} + \text{ASC}_{\text{walk}\_\text{nocar}} \cdot (1 - \text{car\_available}) + \beta_{\text{time\_walk}} \cdot \text{dur\_walking} $$

- **Cycling**:  
  $$ U_{\text{cycle}} = \text{ASC}_{\text{cycle}} \cdot \text{car\_available} + \text{ASC}_{\text{cycle}\_\text{nocar}} \cdot (1 - \text{car\_available}) + + \beta_{\text{time\_cycle}} \cdot \text{dur\_cycling} $$

- **Public Transport**: 
  $$ U_{\text{pt}} = \text{ASC}_{\text{pt}} \cdot \text{car\_available} + \text{ASC}_{\text{pt}\_\text{nocar}} \cdot (1 - \text{car\_available}) + \beta_{\text{cost\_pt}} \cdot \text{cost\_transit} + \beta_{\text{time\_pt}} \cdot \text{dur\_pt\_total} $$

- **Driving**:  
  $$ U_{\text{drive}} = \beta_{\text{cost\_drive}} \cdot \text{cost\_driving\_total} + \beta_{\text{time\_drive}} \cdot \text{dur\_driving} $$

Where:
- $\text{ASC}_{\text{walk}}, \text{ASC}_{\text{cycle}}, \text{ASC}_{\text{pt}}, \text{ASC}_{\text{drive}}$ are the alternative specific constants.
- $\beta_{\text{cost\_walk}}, \beta_{\text{cost\_bike}}, \beta_{\text{cost\_pt}}, \beta_{\text{cost\_drive}}$ are the cost coefficients for walking, cycling, public transport, and driving, respectively.
- $\beta_{\text{log\_dur\_drive}}$ is the coefficient for the non-linear transformation (logarithm) of the driving duration.
- $\beta_{\text{time}}$ is the common time coefficient for all modes.
- $\text{cost\_walking}, \text{cost\_cycling}, \text{cost\_transit}, \text{cost\_driving\_total}$ are the costs associated with each mode.
- $\text{dur\_walking}, \text{dur\_cycling}, \text{dur\_pt\_total}, \text{dur\_driving}$ are the travel durations for each mode.
- The logarithmic transformation of driving duration is represented by $\log(\text{dur\_driving} + 1)$ to ensure the argument inside the log function is always positive.

In [11]:

# Calculate the total public transport duration and total driving cost
df['dur_pt_total'] = df['dur_pt_access'] + df['dur_pt_rail'] + df['dur_pt_bus'] + df['dur_pt_int']
df['cost_driving_total'] = df['cost_driving_fuel'] + df['cost_driving_ccharge']

# Create a Biogeme database
database_3 = db.Database('LPMC', df)
globals().update(database_3.variables)

# Create a new transformed variable for time
# log_dur_walking = database.DefineVariable('log_dur_walking', log(dur_driving))
# log_dur_cycling = database.DefineVariable('log_dur_cycling', log(dur_cycling))
# log_dur_pt = database.DefineVariable('log_dur_pt', log(dur_pt_total))
# log_dur_driving = database.DefineVariable('log_dur_driving', log(dur_driving))


# Define parameters for the utility functions
ASC_WALK = Beta('ASC_WALK', 0, None, None, 0)
ASC_BIKE = Beta('ASC_BIKE', 0, None, None, 0)
ASC_PT = Beta('ASC_PT', 0, None, None, 0)

# Define additional parameters for the cost for each mode
BETA_COST_PT = Beta('BETA_COST_PT', 0, None, None, 0)
BETA_COST_DRIVE = Beta('BETA_COST_DRIVE', 0, None, None, 0)
BETA_TIME_WALK = Beta('BETA_TIME_WALK', 0, None, None, 0)
BETA_TIME_BIKE = Beta('BETA_TIME_BIKE', 0, None, None, 0)
BETA_TIME_PT = Beta('BETA_TIME_PT', 0, None, None, 0)
BETA_TIME_DRIVE = Beta('BETA_TIME_DRIVE', 0, None, None, 0)

# Utility functions with interactions
V1 = ASC_WALK * car_available + ASC_WALK_NOCAR * (1 - car_available) + BETA_TIME_WALK * log(dur_walking)
V2 = ASC_BIKE * car_available + ASC_BIKE_NOCAR * (1 - car_available) + BETA_TIME_BIKE * log(dur_cycling)
V3 = ASC_PT * car_available + ASC_PT_NOCAR * (1 - car_available) + BETA_COST_PT * cost_transit + BETA_TIME_PT * log(dur_pt_total)
V4 = BETA_COST_DRIVE * cost_driving_total + BETA_TIME_DRIVE * log(dur_driving)

# Define the model
logprob_3 = models.loglogit(V, None, travel_mode)

# Estimate the model
biogeme_3 = bio.BIOGEME(database_3, logprob_3)
biogeme_3.modelName = 'Model_3'
results_model_3 = biogeme_3.estimate()

# Output
print(results_model_3.getEstimatedParameters())


                    Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_BIKE        -3.400561      0.170147   -19.986058  0.000000e+00
ASC_BIKE_NOCAR  -0.660073      0.191846    -3.440647  5.803257e-04
ASC_PT          -1.286285      0.087745   -14.659354  0.000000e+00
ASC_PT_NOCAR     1.891402      0.129159    14.643935  0.000000e+00
ASC_WALK         1.321402      0.134678     9.811583  0.000000e+00
ASC_WALK_NOCAR   4.011227      0.185470    21.627380  0.000000e+00
BETA_COST_DRIVE -0.167348      0.018986    -8.814246  0.000000e+00
BETA_COST_PT    -0.225000      0.034697    -6.484687  8.891621e-11
BETA_TIME_BIKE  -5.193837      0.465663   -11.153645  0.000000e+00
BETA_TIME_DRIVE -6.504902      0.440982   -14.750963  0.000000e+00
BETA_TIME_PT    -3.243180      0.277499   -11.687180  0.000000e+00
BETA_TIME_WALK  -8.098924      0.406667   -19.915352  0.000000e+00


In [12]:
# Get general statistics for Model 3
general_stats_model_3 = results_model_3.getGeneralStatistics()

# Extract AIC and BIC for Model 3
aic_model_3 = general_stats_model_3['Akaike Information Criterion'][0]
bic_model_3 = general_stats_model_3['Bayesian Information Criterion'][0]

print("Model 3 - AIC:", aic_model_3, "BIC:", bic_model_3)

Model 3 - AIC: 7425.979324821896 BIC: 7504.185643118891


### Model 4

#### Nesting

In [13]:

# Calculate the total public transport duration and total driving cost
df['dur_pt_total'] = df['dur_pt_access'] + df['dur_pt_rail'] + df['dur_pt_bus'] + df['dur_pt_int']
df['cost_driving_total'] = df['cost_driving_fuel'] + df['cost_driving_ccharge']

# Create a Biogeme database
database_4_nest = db.Database('LPMC', df)
globals().update(database_4_nest.variables)

# Define parameters for the utility functions
ASC_WALK = Beta('ASC_WALK', 0, None, None, 0)
ASC_BIKE = Beta('ASC_BIKE', 0, None, None, 0)
ASC_PT = Beta('ASC_PT', 0, None, None, 0)
ASC_DRIVE = Beta('ASC_DRIVE', 0, None, None, 0)

# Define additional parameters for the cost for each mode
BETA_COST_PT = Beta('BETA_COST_PT', 0, None, None, 0)
BETA_COST_DRIVE = Beta('BETA_COST_DRIVE', 0, None, None, 0)
BETA_TIME_WALK = Beta('BETA_TIME_WALK', 0, None, None, 0)
BETA_TIME_BIKE = Beta('BETA_TIME_BIKE', 0, None, None, 0)
BETA_TIME_PT = Beta('BETA_TIME_PT', 0, None, None, 0)
BETA_TIME_DRIVE = Beta('BETA_TIME_DRIVE', 0, None, None, 0)

# New parameters for interactions
BETA_DRIVE_CAROWN = Beta('BETA_DRIVE_CAROWN', 0, None, None, 0)
BETA_COST_DRIVE_CAROWN = Beta('BETA_COST_DRIVE_CAROWN', 0, None, None, 0)
BETA_TIME_DRIVE_CAROWN = Beta('BETA_TIME_DRIVE_CAROWN', 0, None, None, 0)
BETA_PT_INT = Beta('BETA_PT_INT', 0, None, None, 0)
BETA_COST_PT_INT = Beta('BETA_COST_PT_INT', 0, None, None, 0)
BETA_TIME_PT_INT = Beta('BETA_TIME_INT', 0, None, None, 0)
BETA_LOG_DUR_DRIVE = Beta('BETA_LOG_DUR_DRIVE', 0, None, None, 0)

# Utility functions with interactions
V1 = ASC_WALK * car_available + ASC_WALK_NOCAR * (1 - car_available) + BETA_TIME_WALK * dur_walking
V2 = ASC_BIKE * car_available + ASC_BIKE_NOCAR * (1 - car_available) + BETA_TIME_BIKE * dur_cycling
V3 = ASC_PT * car_available + ASC_PT_NOCAR * (1 - car_available) + BETA_COST_PT * cost_transit + BETA_TIME_PT * dur_pt_total
V4 = BETA_COST_DRIVE * cost_driving_total + BETA_TIME_DRIVE * dur_driving

# Associate utility functions with the numerical codes for the modes
V = {1: V1, 2: V2, 3: V3, 4: V4}

# Define nest coefficients
MOTOR = Beta('MOTOR', 1, 1, None, 0)  # Nest parameter for motorized transport
NON_MOTOR = Beta('NON_MOTOR', 1, 1, None, 0)  # Nest parameter for non-motorized transport

# Define nests
# Assuming that alternatives are coded as: 1 for walking, 2 for cycling, 3 for public transport, and 4 for driving
nest_motorized = MOTOR, [3, 4]  # Nest for public transport (3) and driving (4)
nest_non_motorized = NON_MOTOR, [1, 2]  # Nest for walking (1) and cycling (2)

# Combine nests into a list
nests = nest_motorized, nest_non_motorized

# Define the nested logit model
nested_logit = models.nested(V, None, nests, travel_mode)

# Estimate the model
biogeme_4_nest = bio.BIOGEME(database_4_nest, nested_logit)
biogeme_4_nest.modelName = 'Model_4_nest'
results_model_4 = biogeme_4_nest.estimate()

# Print the estimation results
print(results_model_4.getEstimatedParameters())

                     Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_BIKE         -9.989938      0.338083   -29.548772      0.000000
ASC_BIKE_NOCAR   -6.065947      1.325303    -4.577028      0.000005
ASC_PT           -0.796232      0.426693    -1.866054      0.062034
ASC_PT_NOCAR      4.365088      2.252706     1.937708      0.052659
ASC_WALK          8.300606      0.993290     8.356682      0.000000
ASC_WALK_NOCAR   16.114658      1.166221    13.817844      0.000000
BETA_COST_DRIVE  -0.200783      0.105806    -1.897645      0.057743
BETA_COST_PT     -0.215942      0.130702    -1.652166      0.098501
BETA_TIME_BIKE   -7.481120      4.874134    -1.534861      0.124818
BETA_TIME_DRIVE -11.970558      6.442910    -1.857943      0.063177
BETA_TIME_PT     -7.469432      3.923552    -1.903742      0.056944
BETA_TIME_WALK  -40.449408      2.592351   -15.603371      0.000000
MOTOR            18.164813      9.192781     1.975987      0.048156
NON_MOTOR         4.138961      0.465310     8.8

In [14]:
general_stats_model_4 = results_model_4.getGeneralStatistics()

# Extract AIC and BIC for Model 4
aic_model_4 = general_stats_model_4['Akaike Information Criterion'][0]
bic_model_4 = general_stats_model_4['Bayesian Information Criterion'][0]

print("Model 4 - AIC:", aic_model_4, "BIC:", bic_model_4)

Model 4 - AIC: -7153.593884007429 BIC: -7062.3531793276015


In [15]:
LR_test = 2 * (results_model_4.data.logLike - results_model_2.data.logLike)
print(LR_test)
x_qhi = st.chi2.sf(LR_test, 4)
x_qhi = st.chi2.ppf(0.05, 4)
print(x_qhi)

final_log_likelihood = general_stats_model_4['Final log likelihood'][0]
print(f"Final log-likelihood: {final_log_likelihood}")

14583.573208829324
0.7107230213973239
Final log-likelihood: 3590.7969420037143


#### Cross nesting

In [16]:

# Calculate the total public transport duration and total driving cost
df['dur_pt_total'] = df['dur_pt_access'] + df['dur_pt_rail'] + df['dur_pt_bus'] + df['dur_pt_int']
df['cost_driving_total'] = df['cost_driving_fuel'] + df['cost_driving_ccharge']

# Create a Biogeme database
database_4_crossnest = db.Database('LPMC', df)
globals().update(database_4_crossnest.variables)

# Define parameters for the utility functions
ASC_WALK = Beta('ASC_WALK', 0, None, None, 0)
ASC_BIKE = Beta('ASC_BIKE', 0, None, None, 0)
ASC_PT = Beta('ASC_PT', 0, None, None, 0)
ASC_DRIVE = Beta('ASC_DRIVE', 0, None, None, 0)

# Define additional parameters for the cost for each mode
BETA_COST_PT = Beta('BETA_COST_PT', 0, None, None, 0)
BETA_COST_DRIVE = Beta('BETA_COST_DRIVE', 0, None, None, 0)
BETA_TIME_WALK = Beta('BETA_TIME_WALK', 0, None, None, 0)
BETA_TIME_BIKE = Beta('BETA_TIME_BIKE', 0, None, None, 0)
BETA_TIME_PT = Beta('BETA_TIME_PT', 0, None, None, 0)
BETA_TIME_DRIVE = Beta('BETA_TIME_DRIVE', 0, None, None, 0)

# New parameters for interactions
BETA_DRIVE_CAROWN = Beta('BETA_DRIVE_CAROWN', 0, None, None, 0)
BETA_COST_DRIVE_CAROWN = Beta('BETA_COST_DRIVE_CAROWN', 0, None, None, 0)
BETA_TIME_DRIVE_CAROWN = Beta('BETA_TIME_DRIVE_CAROWN', 0, None, None, 0)
BETA_PT_INT = Beta('BETA_PT_INT', 0, None, None, 0)
BETA_COST_PT_INT = Beta('BETA_COST_PT_INT', 0, None, None, 0)
BETA_TIME_PT_INT = Beta('BETA_TIME_INT', 0, None, None, 0)
BETA_LOG_DUR_DRIVE = Beta('BETA_LOG_DUR_DRIVE', 0, None, None, 0)

# Utility functions with interactions
V1 = ASC_WALK * car_available + ASC_WALK_NOCAR * (1 - car_available) + BETA_TIME_WALK * dur_walking
V2 = ASC_BIKE * car_available + ASC_BIKE_NOCAR * (1 - car_available) + BETA_TIME_BIKE * dur_cycling
V3 = ASC_PT * car_available + ASC_PT_NOCAR * (1 - car_available) + BETA_COST_PT * cost_transit + BETA_TIME_PT * dur_pt_total
V4 = BETA_COST_DRIVE * cost_driving_total + BETA_TIME_DRIVE * dur_driving

# Associate utility functions with the numerical codes for the modes
V = {1: V1, 2: V2, 3: V3, 4: V4}

# Define nest coefficients
MOTOR = Beta('MOTOR', 1, 1, None, 0)  # Nest parameter for motorized transport
PRIVATIZED = Beta('PRIVATIZED', 1, 1, None, 0)  # Nest parameter for non-motorized transport

# Define nests
# Assuming that alternatives are coded as: 1 for walking, 2 for cycling, 3 for public transport, and 4 for driving
alpha = 0.5
nest_motorized = MOTOR, {1: 0,
                         2: 0,
                         3: 1,
                         4: 1 - alpha}
nest_privatized = PRIVATIZED, {1: 1,
                               2: 1,
                               3: 0,
                               4: alpha}

# Combine nests into a list
nests = nest_motorized, nest_privatized

# Define the cross-nested logit model
crossnested_logit = models.logcnl(V, None, nests, travel_mode)

# Estimate the model
biogeme_4_crossnest = bio.BIOGEME(database_4_crossnest, crossnested_logit)
biogeme_4_crossnest.modelName = 'Model_4_crossnest'
results_model_4_cross = biogeme_4_crossnest.estimate()

# Print the estimation results
print(results_model_4_cross.getEstimatedParameters())

                    Value  Active bound  Rob. Std err  Rob. t-test  \
ASC_BIKE        -3.373720           0.0      0.433606    -7.780615   
ASC_BIKE_NOCAR  -1.167858           0.0      0.223454    -5.226381   
ASC_PT          -0.891271           0.0      0.067370   -13.229431   
ASC_PT_NOCAR     1.212475           0.0      0.214739     5.646267   
ASC_WALK         1.251313           0.0      0.195960     6.385553   
ASC_WALK_NOCAR   3.455841           0.0      0.383296     9.016122   
BETA_COST_DRIVE -0.117419           0.0      0.016485    -7.122615   
BETA_COST_PT    -0.130442           0.0      0.028221    -4.622118   
BETA_TIME_BIKE  -3.463590           0.0      0.496660    -6.973761   
BETA_TIME_DRIVE -4.031347           0.0      0.573585    -7.028329   
BETA_TIME_PT    -1.973495           0.0      0.330463    -5.971918   
BETA_TIME_WALK  -7.402958           0.0      0.730793   -10.130039   
MOTOR            2.531993           0.0      0.601108     4.212212   
PRIVATIZED       1.0

In [20]:
general_stats_model_4_cross = results_model_4_cross.getGeneralStatistics()

# Extract AIC and BIC for Model 4
aic_model_4 = general_stats_model_4_cross['Akaike Information Criterion'][0]
bic_model_4 = general_stats_model_4_cross['Bayesian Information Criterion'][0]

print("Model 4 - AIC:", aic_model_4, "BIC:", bic_model_4)

Model 4 - AIC: 7418.0466763234535 BIC: 7509.287381003281


In [19]:
LR_test = 2 * (results_model_4_cross.data.logLike - results_model_2.data.logLike)
print(LR_test)
x_qhi = st.chi2.sf(LR_test, 4)
x_qhi = st.chi2.ppf(0.05, 4)
print(x_qhi)


final_log_likelihood = general_stats_model_4['Final log likelihood'][0]
print(f"Final log-likelihood: {final_log_likelihood}")

11.932648498442177
0.7107230213973239
Final log-likelihood: 3590.7969420037143
